## Chapter6
### テキスト解析とチャットボット作成

### Chapter6-1
形態要素解析 Morphological Analysis

形態要素解析とは、自然言語の文章を意味を持つ最小の単位である「形態素」に分割し、品詞を判別する作業。
形態素解析は、機械翻訳や、かな漢字変換、また、テキストマイニングなど様々な分野で利用されている。

日本語の形態要素解析を行うのは工夫が必要。
- _文法規則_による方法
- _確率的言語モデル_を用いる方法 ※最近人気

In [5]:
# MeCab 定番ライブラリー
import MeCab
mecab = MeCab.Tagger() # MeCabオブジェクト生成
malist = mecab.parse('庭には二羽鶏がいる。') # 形態要素解析
print(malist)


庭	ニワ	ニワ	庭	名詞-普通名詞-一般			0
に	ニ	ニ	に	助詞-格助詞			
は	ワ	ハ	は	助詞-係助詞			
二	フタ	フタ	二	名詞-数詞			2
羽	ワ	ワ	羽	接尾辞-名詞的-助数詞			
鶏	ニワトリ	ニワトリ	鶏	名詞-普通名詞-一般			0
が	ガ	ガ	が	助詞-格助詞			
いる	イル	イル	居る	動詞-非自立可能	上一段-ア行	終止形-一般	0
。			。	補助記号-句点			
EOS



In [6]:
# ピュアPython解析器「Janome」
from janome.tokenizer import Tokenizer
t = Tokenizer()
malist = t.tokenize('庭には二羽鶏がいる。')
for n in malist:
    print(n)

庭	名詞,一般,*,*,*,*,庭,ニワ,ニワ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
二	名詞,数,*,*,*,*,二,ニ,ニ
羽	名詞,接尾,助数詞,*,*,*,羽,ワ,ワ
鶏	名詞,一般,*,*,*,*,鶏,ニワトリ,ニワトリ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
いる	動詞,自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。


In [26]:
from janome.tokenizer import Tokenizer
import zipfile
import os.path, urllib.request as req

# 銀河鉄道の夜のZIPファイルをダウンロード
url = 'http://www.aozora.gr.jp/cards/000081/files/456_ruby_145.zip'
local = '456_ruby_145.zip'
if not os.path.exists(local):
    print('ZIPファイルをダウンロード')
    req.urlretrieve(url, local)

# ZIPファイル内のテキストファイルを読み込む
zf = zipfile.ZipFile(local, 'r') # ZIPファイルを読み込む
fp = zf.open('gingatetsudono_yoru.txt', 'r') # アーカイブ内のテキストを読む
bindata = fp.read()
txt = bindata.decode('shift_jis') # テキストがshift_JISなのでデコード

# 形態要素解析オブジェクトの生成
t = Tokenizer()

# テキストを一行ずつ処理
word_dic = {}
lines = txt.split('/r/n')
for line in lines:
    malist = t.tokenize(line)
    for w in malist:
        word = w.surface
        ps = w.part_of_speech # 品詞
        if ps.find('名詞') < 0: continue # 名詞だけカウント：find(***)　⇒ ***が出現する位置を返す, continueは下の処理をせずforの位置に戻ることを指す
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1



In [25]:
# よく使われる単語を表示
keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for i, key in enumerate(keys[:50]):
    print('{0}位：{1}回 {2} '.format(i,key[1],key[0]))

0位：209回 よう 
1位：206回 の 
2位：190回 ジョバンニ 
3位：102回 人 
4位：101回 カムパネルラ 
5位：101回 ん 
6位：66回 方 
7位：65回 中 
8位：59回 ぼく 
9位：57回 それ 
10位：56回 たち 
11位：54回 みんな 
12位：53回 二 
13位：47回 一 
14位：46回 何 
15位：45回 ほんとう 
16位：45回 鳥 
17位：43回 どこ 
18位：39回 こと 
19位：39回 窓 
20位：39回 汽車 
21位：38回 前 
22位：38回 そう 
23位：38回 いま 
24位：37回 眼 
25位：35回 川 
26位：35回 とき 
27位：33回 僕 
28位：32回 もの 
29位：30回 たくさん 
30位：29回 水 
31位：29回 お 
32位：29回 ら 
33位：28回 青年 
34位：27回 銀河 
35位：27回 そこ 
36位：27回 こっち 
37位：27回 さっき 
38位：26回 上 
39位：26回 ろ 
40位：26回 風 
41位：25回 星 
42位：25回 顔 
43位：25回 向う 
44位：25回 女の子 
45位：24回 野原 
46位：23回 お父さん 
47位：23回 天の川 
48位：22回 声 
49位：21回 い 


In [28]:
# 夏目漱石の「こころ」もカウントする
from janome.tokenizer import Tokenizer
import os.path, urllib.request as req
import zipfile

url = 'http://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip'
local = '773_ruby_5968.zip'

# ディレクトリ確認
if not os.path.exists(local):
    print('Zipファイルダウンロード')
    req.urlretrieve(url, local)

# ZIPファイル読み込み
zp = zipfile.ZipFile(local)
fp = zp.open('kokoro.txt', 'r')
bindata = fp.read()
txt = bindata.decode('shift_jis')

# 形態要素解析
t = Tokenizer()

# カウント用dict作成
word_dic = {}
lines = txt.split('\r\n')
for line in lines:
    malist = t.tokenize(line)
    for w in malist:
        # 文字だけ
        word = w.surface
        # 品詞
        ps = w.part_of_speech
        if ps.find('名詞') < 0: continue
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1

TypeError: 'int' object is not subscriptable

In [34]:
keys = sorted(word_dic.items(),key=lambda x:x[1], reverse=True) # valueでソート: x[1]
for i, tup in enumerate(keys[:50]):
    print('第{0}位：{1} - {2}回'.format(i+1, tup[0], tup[1]))

第1位：私 - 2700回
第2位：の - 1483回
第3位：先生 - 600回
第4位：事 - 576回
第5位：よう - 523回
第6位：それ - 409回
第7位：もの - 393回
第8位：人 - 390回
第9位：奥さん - 388回
第10位：時 - 379回
第11位：彼 - 314回
第12位：父 - 272回
第13位：自分 - 264回
第14位：二 - 263回
第15位：中 - 259回
第16位：何 - 251回
第17位：一 - 249回
第18位：ん - 241回
第19位：うち - 238回
第20位：い - 234回
第21位：十 - 201回
第22位：方 - 200回
第23位：あなた - 187回
第24位：母 - 171回
第25位：前 - 168回
第26位：お嬢さん - 166回
第27位：上 - 156回
第28位：気 - 150回
第29位：今 - 150回
第30位：顔 - 135回
第31位：め - 133回
第32位：言葉 - 128回
第33位：ため - 126回
第34位：字 - 124回
第35位：三 - 123回
第36位：日 - 123回
第37位：眼 - 123回
第38位：そこ - 120回
第39位：心 - 116回
第40位：＃ - 115回
第41位：下げ - 113回
第42位：見出し - 113回
第43位：［＃「 - 113回
第44位：５ - 110回
第45位：妻 - 108回
第46位：口 - 107回
第47位：通り - 105回
第48位：お - 105回
第49位：家 - 96回
第50位：間 - 93回
